# Requêtes SQL sur la base de données

In [1]:
import sys

sys.path.append("../")
from db import Engine, Transaction, Transmission, Customer, Model, MySQLConnector

conn = MySQLConnector()
conn.connect()
conn

MySQLConnector(host='localhost', user='admin', password='secret', database='cars24', db=<mysql.connector.connection_cext.CMySQLConnection object at 0x73b48771ada0>)

In [2]:
# Liste des tables
conn.execute("SHOW TABLES")

,Tables_in_cars24
0,customer
1,engine
2,model
3,transaction
4,transmission


In [3]:
# Group by simple
conn.execute(
    """
SELECT sum(capacity) FROM engine
WHERE fuel_type = 'Petrol'
GROUP BY fuel_type
"""
)

,sum(capacity)
0,8270.0


In [4]:
# Merge
conn.execute(
    """
SELECT id_model, model_name 
FROM model
WHERE id_engine IN (
    SELECT id_engine
    FROM engine
    WHERE fuel_type = 'Diesel'
)
LIMIT 10
"""
)

,id_model,model_name
0,00bcf0d5-8fd5-48e0-8df1-7c3903033a68,2018 Maruti Dzire VDI
1,09ff45d8-bb3f-45cf-9e87-d92628b68dab,2015 Maruti Baleno ZETA DIESEL 1.3
2,0deeaafb-1c44-4df7-a16e-ec26d1fad85d,2017 Maruti Baleno ZETA DIESEL 1.3
3,114d99a6-7190-434c-bef6-4b979c1917e5,2015 Maruti Swift Dzire VDI ABS
4,1673c431-f218-4ef3-b157-b354455eefd0,2016 Maruti Ertiga VDI SHVS
5,1da733f2-7f72-401d-af57-0499da8dbbda,2016 Maruti Vitara Brezza VDI
6,1e2b00fd-23b5-41b2-9ee6-6d0244d5c7c5,2017 Maruti Ciaz SIGMA DIESEL 1.3
7,24b40531-904b-4b3f-bdb4-d240cf56ec43,2013 Maruti Swift ZDI
8,2b00c04d-aab8-421a-a5d1-20c4258dd467,2011 Maruti Swift Dzire VDI
9,315ba5ca-1c17-4f81-8374-4bf7080621c1,2018 Maruti Ertiga ZDI + SHVS


In [5]:
conn.execute(
    """
SELECT model_name, price, km_driven, email, rating, fuel_type, capacity FROM transaction 
LEFT JOIN customer ON customer.id_customer = transaction.id_customer
LEFT JOIN model ON model.id_model = transaction.id_model
LEFT JOIN engine ON model.id_engine = engine.id_engine
WHERE model.id_transmission IN (
    SELECT id_transmission
    FROM transmission
    WHERE transmission_type = 'Manual'
)
LIMIT 10
"""
)

,model_name,price,km_driven,email,rating,fuel_type,capacity
0,2016 Maruti Alto 800 LXI,264000.0,17816.0,coneal@example.com,3 star,Petrol,796.0
1,2020 Maruti Vitara Brezza VXI,710000.0,24819.0,lesterrebecca@example.net,2 star,Petrol,1462.0
2,2019 Maruti Baleno DELTA PETROL 1.2,529000.0,65533.0,cherrykatherine@example.net,3 star,Petrol,1197.0
3,2017 Maruti Baleno ALPHA PETROL 1.2,619000.0,81548.0,richardlevy@example.net,4 star,Petrol,1197.0
4,2015 Maruti Swift Dzire VXI,389000.0,82619.0,sabrina77@example.org,4 star,Petrol,1197.0
5,2023 Maruti Alto K10 LXI,435000.0,14943.0,bradley18@example.org,3 star,Petrol,998.0
6,2018 Maruti Celerio VXI (O),408000.0,37192.0,dorr@example.com,1 star,Petrol,998.0
7,2017 Maruti Alto 800 LXI,278000.0,63989.0,lutzlisa@example.com,2 star,Petrol,796.0
8,2015 Maruti Swift Dzire VXI,468000.0,96263.0,elizabeth21@example.net,4 star,Petrol,1197.0
9,2017 Maruti Dzire VDI,613000.0,70959.0,angelafuller@example.com,4 star,Diesel,1248.0


In [6]:
# Créons notre propre moto :)
from db import Model, Engine, Transmission

# Liste des moteurs disponibles
conn.execute("SELECT * FROM engine")

,id_engine,fuel_type,capacity
0,0f5f4c3e-150f-49d4-9e6b-915ee207a43a,Petrol,1196.0
1,2343f91d-5291-47ee-96a4-33b827968fca,Petrol,1197.0
2,3a4c6a69-05c2-4597-95d6-b0f71a474f76,Petrol,1462.0
3,433f8970-ea23-4e07-8294-e82821e11f1d,Petrol,998.0
4,485f23bd-ca7a-4ad2-9d84-4e7908ef60a4,CNG,1197.0
5,53f68bf4-bf17-468d-8c1c-4381e7d47445,Petrol,1373.0
6,58812d8a-7c0b-4786-9686-1ab87d0d8ddb,CNG,796.0
7,5b16ac17-1268-4078-afd1-2aba6819a246,Diesel,1248.0
8,62dfee90-91b1-4b51-9946-55968362a1f6,Petrol,1248.0
9,7b92fd64-a65f-4f79-98c9-ffe4c7a81aca,CNG,1373.0


In [7]:
# Liste des transmission
conn.execute("SELECT * FROM transmission")

,id_transmission,transmission_type
0,81df1b3c-ec67-46bc-9afd-0ef81974be74,Manual
1,aa54da78-124c-4a4e-a240-9b1026a8b453,Automatic


In [9]:
new_model = Model(
    model_name = "Ma super Moto custom",
    manufacturing_year = 2025,
    id_engine = "62dfee90-91b1-4b51-9946-55968362a1f6", # Gazoil avec un gros réservoir
    id_transmission = "81df1b3c-ec67-46bc-9afd-0ef81974be74" # Auto
)
new_model

Model(id_model='8edc69cb-b0df-4a93-83a3-5e1b75a36b49', model_name='Ma super Moto custom', manufacturing_year=2025, id_engine='62dfee90-91b1-4b51-9946-55968362a1f6', id_transmission='81df1b3c-ec67-46bc-9afd-0ef81974be74')

In [10]:
conn.add_entry(new_model)

'8edc69cb-b0df-4a93-83a3-5e1b75a36b49'

In [11]:
conn.execute("SELECT * FROM model WHERE model_name = 'Ma super Moto custom'")

,id_model,model_name,id_engine,id_transmission,manufacturing_year
0,8edc69cb-b0df-4a93-83a3-5e1b75a36b49,Ma super Moto custom,62dfee90-91b1-4b51-9946-55968362a1f6,81df1b3c-ec67-46bc-9afd-0ef81974be74,2025
